In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
import seaborn as sns; sns.set() # prettify matplotlib

import numpy as np
import sklearn.gaussian_process as gp
import GPy

In [ ]:
# local modules
import turbo as tb
import turbo.modules as tm
import turbo.plotting as tp
import turbo.gui as tg

In [ ]:
# make deterministic
np.random.seed(42)

Function to optimize:

In [ ]:
#f = lambda x: 1 * x * np.cos(x)
f = lambda x: 100 * np.sin(x**2/5) * np.cos(x*1.5) + 100
xmin, xmax = 0, 12
xs = np.linspace(xmin, xmax, num=200)

ys = f(xs)
best_y = np.min(ys)
best_x = xs[np.argmin(ys)]

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(xs, ys, 'g-', label='objective')
plt.plot(best_x, best_y, 'bo', label='optima')
plt.legend(loc='upper left')
plt.margins(0.01, 0.1)
plt.xlabel(r'$x$')
plt.ylabel(r'$f(x)$')
plt.show()

# Optimisation

In [ ]:
op = tb.Optimiser(f, 'min', bounds=[('x', xmin, xmax)], pre_phase_trials=6, settings_preset='default')
#op.surrogate_factory = tm.SciKitGPSurrogate(model_params=dict(kernel=1.0*gp.kernels.RBF()+gp.kernels.WhiteKernel(), normalize_y=True))
kernel = GPy.kern.RBF(input_dim=1) + GPy.kern.White(input_dim=1)
#kernel.white.variance.set_prior(GPy.priors.Gamma.from_EV(0.8, 0.4))
op.surrogate = tm.GPySurrogate(model_params=dict(kernel=kernel, noise_var=1e-10, normalizer=True))
op.acq_func_factory = tm.UCB.Factory(beta=2)
rec = tp.PlottingRecorder(op)

In [ ]:
tg.OptimiserProgressBar(op)
op.run(max_trials=25)

In [ ]:
op.get_incumbent()

In [ ]:
tp.plot_error_over_time(rec, true_best=best_y, log_scale=False, fig_ax=plt.subplots(figsize=(8, 3)));

In [ ]:
tp.interactive_plot_trial_1D(rec, param='x', true_objective=f)

# Try optimising the same function with random search

In [ ]:
ro = tb.Optimiser(f, 'min', bounds=[('x', xmin, xmax)], pre_phase_trials=float('inf'), settings_preset='random_search')
rrec = tp.PlottingRecorder(ro)

In [ ]:
tg.OptimiserProgressBar(ro)
ro.run(max_trials=100)

In [ ]:
ro.get_incumbent()

In [ ]:
tp.plot_error_over_time(rrec, true_best=best_y, log_scale=False, fig_ax=plt.subplots(figsize=(8, 3)))

In [ ]:
tp.interactive_plot_trial_1D(rrec, param='x', true_objective=f)